# Consolidação do dataset de notícias rotuladas do mercado financeiro brasileiro

## Importando Libs

In [ ]:
import pandas as pd
from statsmodels.stats.inter_rater import fleiss_kappa

## Importando classificação dos revisores

In [ ]:
engineerV = pd.read_csv('https://raw.githubusercontent.com/vinelouzada/ararafin-research/refs/heads/main/Not%C3%ADcias%20do%20Mercado%20Financeiro%20-%20Sentimentos%20-%20Engenheiro%20V.csv')

In [ ]:
engineerR = pd.read_csv('https://raw.githubusercontent.com/vinelouzada/ararafin-research/refs/heads/main/Not%C3%ADcias%20do%20Mercado%20Financeiro%20-%20Sentimentos%20-%20Engenheiro%20R.csv')

In [ ]:
engineerI = pd.read_csv('http://raw.githubusercontent.com/vinelouzada/ararafin-research/refs/heads/main/Not%C3%ADcias%20do%20Mercado%20Financeiro%20-%20Sentimentos%20-%20Engenheiro%20I.csv')

In [ ]:
investorR = pd.read_csv('https://raw.githubusercontent.com/vinelouzada/ararafin-research/refs/heads/main/Not%C3%ADcias%20do%20Mercado%20Financeiro%20-%20Sentimentos%20-%20Investidor%20R.csv')

## Criando a base do dataset

In [ ]:
base_cols = [
    "_id",
    "title",
    "body",
    "summary",
    "created_at",
    "published_at",
    "source",
    "url"
]

sentiment_cols = [
    "Sentimento esperado para Renda Fixa",
    "Sentimento esperado para FIIs",
    "Sentimento esperado para Ações",
]

In [ ]:
base = engineerV[base_cols]

## Mesclando classificações

In [ ]:
def add_sentiments(base_df, df, source):
    sentiments = (
        df[["_id"] + sentiment_cols]
        .rename(columns={
            "Sentimento esperado para Renda Fixa": f"Sentimento esperado para Renda Fixa_{source}",
            "Sentimento esperado para FIIs": f"Sentimento esperado para FIIs_{source}",
            "Sentimento esperado para Ações": f"Sentimento esperado para Ações_{source}",
        })
    )

    return base_df.merge(sentiments, on="_id", how="left")

In [ ]:
base = add_sentiments(base, engineerV, "engineerV")
base = add_sentiments(base, engineerR, "engineerR")
base = add_sentiments(base, engineerI, "engineerI")
base = add_sentiments(base, investorR, "investorR")

## Análise de concordância dos avaliadores

In [ ]:
renda_fixa_cols = [
    'Sentimento esperado para Renda Fixa_engineerV',
    'Sentimento esperado para Renda Fixa_engineerR',
    'Sentimento esperado para Renda Fixa_engineerI',
    'Sentimento esperado para Renda Fixa_investorR'
]

fiis_cols = [
    'Sentimento esperado para FIIs_engineerV',
    'Sentimento esperado para FIIs_engineerR',
    'Sentimento esperado para FIIs_engineerI',
    'Sentimento esperado para FIIs_investorR'
]

acoes_cols = [
    'Sentimento esperado para Ações_engineerV',
    'Sentimento esperado para Ações_engineerR',
    'Sentimento esperado para Ações_engineerI',
    'Sentimento esperado para Ações_investorR'
]

### Calculando Fleiss’ Kappa
O Fleiss’ Kappa serve para medir o grau de concordância entre vários avaliadores (mais de 2)

In [ ]:
def calculate_fleiss_kappa(classifications):
    matrix = classifications.apply(lambda line: line.value_counts(), axis=1).fillna(0).astype(int)
    kappa = fleiss_kappa(matrix.values)
    return round(kappa, 2)

#### Grau de concordância da classificação sob o viés de FIIs

In [ ]:
calculate_fleiss_kappa(base[fiis_cols])

np.float64(0.56)

#### Grau de concordância da classificação sob o viés de Ações


In [ ]:
calculate_fleiss_kappa(base[acoes_cols])

np.float64(0.57)

#### Grau de concordância da classificação sob o viés de Renda Fixa


In [ ]:
calculate_fleiss_kappa(base[renda_fixa_cols])

np.float64(0.35)

## Calculando moda
Calculando a moda com o critério de desempate sendo a classificação do investidor

In [ ]:
def expected_sentiment_with_investor_tiebreak(row, cols, investor_col):
    counts = row[cols].value_counts()

    if len(counts) == 0:
        return None

    top_count = counts.iloc[0]

    tied = counts[counts == top_count].index.tolist()

    if len(tied) == 1:
        return tied[0]

    investor_value = row[investor_col]
    if investor_value in tied:
        return investor_value

    return tied[0]

In [ ]:
base['Sentimento esperado para Renda Fixa'] = base.apply(
    expected_sentiment_with_investor_tiebreak,
    axis=1,
    cols=renda_fixa_cols,
    investor_col='Sentimento esperado para Renda Fixa_investorR'
)

In [ ]:
base['Sentimento esperado para FIIs'] = base.apply(
    expected_sentiment_with_investor_tiebreak,
    axis=1,
    cols=fiis_cols,
    investor_col='Sentimento esperado para FIIs_investorR'
)

In [ ]:
base['Sentimento esperado para Ações'] = base.apply(
    expected_sentiment_with_investor_tiebreak,
    axis=1,
    cols=acoes_cols,
    investor_col='Sentimento esperado para Ações_investorR'
)

### Visualização dos resultados por tipo de investimento

#### FIIs

In [ ]:
base[
    fiis_cols
    + ['Sentimento esperado para FIIs']
].tail()

,Sentimento esperado para FIIs_engineerV,Sentimento esperado para FIIs_engineerR,Sentimento esperado para FIIs_engineerI,Sentimento esperado para FIIs_investorR,Sentimento esperado para FIIs
623,NEUTRO,NEUTRO,NEGATIVO,NEGATIVO,NEGATIVO
624,NEUTRO,NEUTRO,NEUTRO,NEUTRO,NEUTRO
625,POSITIVO,POSITIVO,NEUTRO,POSITIVO,POSITIVO
626,POSITIVO,POSITIVO,POSITIVO,POSITIVO,POSITIVO
627,NEUTRO,NEUTRO,NEUTRO,NEUTRO,NEUTRO


#### Ações

In [ ]:
base[
    acoes_cols
    + ['Sentimento esperado para Ações']
].tail()

,Sentimento esperado para Ações_engineerV,Sentimento esperado para Ações_engineerR,Sentimento esperado para Ações_engineerI,Sentimento esperado para Ações_investorR,Sentimento esperado para Ações
623,NEUTRO,NEUTRO,NEGATIVO,NEGATIVO,NEGATIVO
624,POSITIVO,POSITIVO,POSITIVO,NEUTRO,POSITIVO
625,POSITIVO,POSITIVO,POSITIVO,POSITIVO,POSITIVO
626,POSITIVO,POSITIVO,POSITIVO,POSITIVO,POSITIVO
627,NEUTRO,NEUTRO,POSITIVO,NEUTRO,NEUTRO


#### Renda Fixa

In [ ]:
base[
    renda_fixa_cols
    + ['Sentimento esperado para Renda Fixa']
].tail()

,Sentimento esperado para Renda Fixa_engineerV,Sentimento esperado para Renda Fixa_engineerR,Sentimento esperado para Renda Fixa_engineerI,Sentimento esperado para Renda Fixa_investorR,Sentimento esperado para Renda Fixa
623,NEUTRO,NEUTRO,NEGATIVO,POSITIVO,NEUTRO
624,NEUTRO,NEUTRO,NEUTRO,NEUTRO,NEUTRO
625,NEGATIVO,NEGATIVO,NEUTRO,NEGATIVO,NEGATIVO
626,NEGATIVO,NEGATIVO,POSITIVO,NEUTRO,NEGATIVO
627,NEUTRO,NEUTRO,NEUTRO,NEUTRO,NEUTRO


## Gerando o dataset oficial

In [ ]:
cols_to_drop = renda_fixa_cols + fiis_cols + acoes_cols
base = base.drop(columns=cols_to_drop)

In [ ]:
base.to_csv("analise_sentimentos_textos_financeiros_gabarito_dataset.csv", index=False)